In [1]:
import pandas as pd
import numpy as np

def read_cli_csv(csv_path):
    df_raw = pd.read_csv(csv_path, header=None, skipinitialspace=True)
    df = df_raw[[4, 22, 24]].copy()
    df.columns = ['REF_AREA','TIME_PERIOD','OBS_VALUE']
    df.dropna(subset=['TIME_PERIOD','OBS_VALUE'], inplace=True)
    df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'], format='%Y-%m', errors='coerce')
    df.dropna(subset=['TIME_PERIOD'], inplace=True)
    df['OBS_VALUE'] = pd.to_numeric(df['OBS_VALUE'], errors='coerce')
    df.dropna(subset=['OBS_VALUE'], inplace=True)
    df.sort_values(['REF_AREA','TIME_PERIOD'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# Remove outliers based on rolling MAD and interpolate them
def remove_outliers_rolling_mad(series, window=12, threshold=3.5):
    s = series.copy()
    def rolling_mad(x):
        m = np.median(x)
        return np.median(np.abs(x - m))
    rolling_median = s.rolling(window=window, center=True).median()
    rolling_mad_series = s.rolling(window=window, center=True).apply(rolling_mad, raw=True)
    med_vals = rolling_median.values
    mad_vals = rolling_mad_series.values
    outlier_mask = np.zeros(len(s), dtype=bool)
    for i in range(len(s)):
        if not np.isnan(med_vals[i]) and not np.isnan(mad_vals[i]) and mad_vals[i] != 0:
            if abs(s.iloc[i] - med_vals[i]) > threshold * mad_vals[i]:
                outlier_mask[i] = True
    s[outlier_mask] = np.nan
    s = s.interpolate(method='linear', limit_direction='both')
    return s

# Compute a centered moving average
def moving_average(x, w):
    return x.rolling(window=w, center=True).mean()

# Find local peaks/troughs using a window span
def find_local_extrema(ser, span=5):
    vals = ser.values
    idx = ser.index
    n = len(vals)
    out = []
    for i in range(span, n - span):
        seg = vals[i-span:i+span+1]
        c = vals[i]
        m1 = True
        m2 = True
        for j in range(len(seg)):
            if seg[j] > c: m1 = False
            if seg[j] < c: m2 = False
        if m1:
            out.append((i, 'peak'))
        elif m2:
            out.append((i, 'trough'))
    res = []
    for pt in out:
        if not res:
            res.append(pt)
        else:
            pr_i, pr_tp = res[-1]
            cu_i, cu_tp = pt
            if cu_tp == pr_tp:
                if pr_tp == 'peak':
                    if vals[cu_i] >= vals[pr_i]:
                        res[-1] = pt
                else:
                    if vals[cu_i] <= vals[pr_i]:
                        res[-1] = pt
            else:
                res.append(pt)
    return [(idx[p], t) for (p, t) in res]

# Enforce minimum phase and cycle duration rules
def enforce_phase_rules(tps, series, min_phase=9, min_cycle=24):
    if not tps: return []
    idx_map = {dt: i for i, dt in enumerate(series.index)}
    refined = []
    for pt in tps:
        if not refined:
            refined.append(pt)
        else:
            p_i, p_tp = refined[-1]
            c_i, c_tp = pt
            if p_tp == c_tp:
                dist = abs(idx_map[c_i] - idx_map[p_i])
                if dist < min_cycle:
                    if p_tp == 'peak':
                        if series.loc[c_i] > series.loc[p_i]:
                            refined[-1] = pt
                    else:
                        if series.loc[c_i] < series.loc[p_i]:
                            refined[-1] = pt
                else:
                    refined.append(pt)
            else:
                dist = abs(idx_map[c_i] - idx_map[p_i])
                if dist < min_phase:
                    if p_tp == 'peak':
                        if series.loc[c_i] > series.loc[p_i]:
                            refined[-1] = pt
                    else:
                        if series.loc[c_i] < series.loc[p_i]:
                            refined[-1] = pt
                else:
                    refined.append(pt)
    return refined

# Refine final turning points by searching nearby range in the original series
def search_nearby_final(series, tps, window=4):
    res = []
    idx_map = {dt: i for i, dt in enumerate(series.index)}
    for dt, tp in tps:
        c_i = idx_map[dt]
        left = max(0, c_i - window)
        right = min(len(series) - 1, c_i + window)
        seg = series.iloc[left:right+1]
        if tp == 'peak':
            mx_i = seg.idxmax()
            res.append((mx_i, 'peak'))
        else:
            mn_i = seg.idxmin()
            res.append((mn_i, 'trough'))
    final = []
    for pt in res:
        if not final:
            final.append(pt)
        else:
            p_i, p_tp = final[-1]
            c_i, c_tp = pt
            if p_tp == c_tp:
                if p_tp == 'peak':
                    if series.loc[c_i] >= series.loc[p_i]:
                        final[-1] = pt
                else:
                    if series.loc[c_i] <= series.loc[p_i]:
                        final[-1] = pt
            else:
                final.append(pt)
    return final

# Filter out peaks below 100 or troughs above 100
def last_filter_gt100_lt100(series, tps):
    out = []
    for dt, tp in tps:
        val = series.loc[dt]
        if tp == 'peak':
            if val > 100:
                out.append((dt, tp))
        else:
            if val < 100:
                out.append((dt, tp))
    return out

# Bry-Boschan with two levels of smoothing
def bry_boschan_classic(series, min_phase=9, min_cycle=24):
    long_ma = moving_average(series, 12)
    coarse = find_local_extrema(long_ma, 5)
    coarse_refined = enforce_phase_rules(coarse, long_ma, min_phase, min_cycle)
    short_ma = moving_average(series, 3)
    near_short = []
    idx_map = {dt: i for i, dt in enumerate(short_ma.index)}
    for dt, tp in coarse_refined:
        if dt not in idx_map:
            continue
        i0 = idx_map[dt]
        left = max(0, i0 - 5)
        right = min(len(short_ma)-1, i0 + 5)
        seg = short_ma.iloc[left:right+1]
        if tp == 'peak':
            di = seg.idxmax()
            near_short.append((di, 'peak'))
        else:
            di = seg.idxmin()
            near_short.append((di, 'trough'))
    near_short.sort(key=lambda x: x[0])
    near_short2 = enforce_phase_rules(near_short, short_ma, min_phase, min_cycle)
    final_candidates = search_nearby_final(series, near_short2, 4)
    final_candidates.sort(key=lambda x: x[0])
    final_refined = enforce_phase_rules(final_candidates, series, min_phase, min_cycle)
    filtered = last_filter_gt100_lt100(series, final_refined)
    return filtered

def main():
    csv_file = "CLI.csv"
    df_all = read_cli_csv(csv_file)
    if df_all.empty:
        return
    MIN_PHASE = 9
    MIN_CYCLE = 24
    country_list = df_all['REF_AREA'].dropna().unique()
    for c in country_list:
        sub = df_all[df_all['REF_AREA'] == c]
        if sub.empty:
            continue
        s = sub.set_index('TIME_PERIOD')['OBS_VALUE'].sort_index()
        s_clean = remove_outliers_rolling_mad(s, 12, 3.5)
        tps_final = bry_boschan_classic(s_clean, MIN_PHASE, MIN_CYCLE)
        print(f"\n--- {c} ---")
        if tps_final:
            for (dt, tp) in tps_final:
                print(dt.strftime('%Y-%m'), tp)
        else:
            print("No turning points found.")

if __name__ == "__main__":
    main()



--- A5M ---
1979-03 peak
1983-03 trough
1985-06 peak
1987-02 trough
1988-10 peak
1990-02 trough
1991-04 peak
1993-09 trough
1996-12 peak
1998-08 trough
2000-09 peak
2002-01 trough
2007-12 peak
2009-03 trough
2011-08 peak
2013-11 peak
2016-09 trough
2018-03 peak
2020-06 trough
2024-01 peak

--- AUS ---
1960-06 peak
1961-10 trough
1965-02 peak
1966-04 trough
1968-02 trough
1970-04 peak
1972-09 trough
1973-11 peak
1974-11 trough
1976-08 peak
1978-02 trough
1981-08 peak
1983-04 trough
1985-07 peak
1986-09 trough
1989-07 peak
1992-01 trough
1994-07 peak
1996-11 trough
1999-12 peak
2001-03 trough
2002-06 peak
2003-04 trough
2004-02 peak
2006-05 trough
2008-02 peak
2011-02 trough
2012-05 peak
2015-05 trough
2018-05 peak
2020-05 trough
2022-11 peak
2024-08 trough

--- BRA ---
1997-04 peak
1999-06 trough
2000-11 peak
2003-08 trough
2004-10 peak
2006-05 trough
2008-05 peak
2009-04 trough
2011-04 peak
2012-02 trough
2014-01 peak
2016-08 trough
2024-08 peak

--- CAN ---
1962-02 peak
1963-06 troug